In [2]:
!which python
!which python3.5
!python --version
!python3.5 --version

/usr/local/envs/py3env/bin/python
/usr/local/envs/py3env/bin/python3.5
Python 3.5.6 :: Anaconda, Inc.
Python 3.5.6 :: Anaconda, Inc.


In [3]:
BUCKET='hohukelkazan'

import os
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.5'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3.5'
print(os.environ['PYSPARK_PYTHON'])
print(os.environ['PYSPARK_DRIVER_PYTHON'])

/usr/bin/python3.5
/usr/bin/python3.5


In [4]:
!echo $PYSPARK_PYTHON
!echo $PYSPARK_DRIVER_PYTHON

/usr/bin/python3.5
/usr/bin/python3.5


# Prepare

以下の流れはベイズ分類の時と変わらない

1. トレーニング日に関するデータのcsvの読み込み(スキーマ: header)
2. 読み込んだトレーニング日に関するデータをSpark SQLビューに変換
3. 飛行データのスキーマ定義
4. 飛行データのcsvの読み込み(スキーマ: 3で読み込んだやつ)
5. 読み込んだ飛行データをSpark SQLビューに変換
6. 2,5の結果をjoinさせてトレーニングデータ作成


In [5]:
# read
traindays = spark.read \
    .option("header", "true") \
    .csv('gs://{}/flights/trainday.csv'.format(BUCKET))

In [6]:
# Spark SQLビューに変換
traindays.createOrReplaceTempView('traindays')

In [7]:
# スキーマ定義(TAXI_OUTも数値型にする)
from pyspark.sql.types import StringType, FloatType, StructType, StructField

header = 'FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,ARR_AIRPORT_TZOFFSET,EVENT,NOTIFY_TIME'

def get_structfield(colname):
  if colname in ['ARR_DELAY', 'DEP_DELAY', 'DISTANCE', 'TAXI_OUT']:
    return StructField(colname, FloatType(), True)
  else:
    return StructField(colname, StringType(), True)
  
schema = StructType([get_structfield(colname) for colname in header.split(',')])
print(schema)

StructType(List(StructField(FL_DATE,StringType,true),StructField(UNIQUE_CARRIER,StringType,true),StructField(AIRLINE_ID,StringType,true),StructField(CARRIER,StringType,true),StructField(FL_NUM,StringType,true),StructField(ORIGIN_AIRPORT_ID,StringType,true),StructField(ORIGIN_AIRPORT_SEQ_ID,StringType,true),StructField(ORIGIN_CITY_MARKET_ID,StringType,true),StructField(ORIGIN,StringType,true),StructField(DEST_AIRPORT_ID,StringType,true),StructField(DEST_AIRPORT_SEQ_ID,StringType,true),StructField(DEST_CITY_MARKET_ID,StringType,true),StructField(DEST,StringType,true),StructField(CRS_DEP_TIME,StringType,true),StructField(DEP_TIME,StringType,true),StructField(DEP_DELAY,FloatType,true),StructField(TAXI_OUT,FloatType,true),StructField(WHEELS_OFF,StringType,true),StructField(WHEELS_ON,StringType,true),StructField(TAXI_IN,StringType,true),StructField(CRS_ARR_TIME,StringType,true),StructField(ARR_TIME,StringType,true),StructField(ARR_DELAY,FloatType,true),StructField(CANCELLED,StringType,true),

In [8]:
inputs = 'gs://{}/flights/tzcorr/all_flights-00000-*'.format(BUCKET) # 1/30th
#inputs = 'gs://{}/flights/tzcorr/all_flights-*'.format(BUCKET)
flights_csv = spark.read\
            .schema(schema)\
            .csv(inputs)
# tmpビュー作成
flights_csv.createOrReplaceTempView('flights')

In [9]:
# create train data
traindayquery = """
select
  f.*
from flights f
join traindays t
on f.FL_DATE == t.FL_DATE
where
  t.is_train_day == 'True'
"""
traindata = spark.sql(traindayquery)

In [10]:
# check: 'ARR_DELAY', 'DEP_DELAY', 'DISTANCE', 'TAXI_OUT'がfloatになっている
print(traindata.head(1)) 

[Row(FL_DATE='2015-10-27', UNIQUE_CARRIER='DL', AIRLINE_ID='19790', CARRIER='DL', FL_NUM='1081', ORIGIN_AIRPORT_ID='10397', ORIGIN_AIRPORT_SEQ_ID='1039705', ORIGIN_CITY_MARKET_ID='30397', ORIGIN='ATL', DEST_AIRPORT_ID='14027', DEST_AIRPORT_SEQ_ID='1402702', DEST_CITY_MARKET_ID='34027', DEST='PBI', CRS_DEP_TIME='2015-10-27 23:17:00', DEP_TIME='2015-10-27 23:18:00', DEP_DELAY=1.0, TAXI_OUT=15.0, WHEELS_OFF='2015-10-27 23:33:00', WHEELS_ON='2015-10-28 00:51:00', TAXI_IN='5.00', CRS_ARR_TIME='2015-10-28 01:06:00', ARR_TIME='2015-10-28 00:56:00', ARR_DELAY=-10.0, CANCELLED='0.00', CANCELLATION_CODE=None, DIVERTED='0.00', DISTANCE=545.0, DEP_AIRPORT_LAT='33.63666667', DEP_AIRPORT_LON='-84.42777778', DEP_AIRPORT_TZOFFSET='-14400.0', ARR_AIRPORT_LAT='26.68305556', ARR_AIRPORT_LON='-80.09555556', ARR_AIRPORT_TZOFFSET='-14400.0', EVENT=None, NOTIFY_TIME=None)]


In [11]:
# describe()メソッドは列ごとの統計情報を計算して、show()はその結果を表示するよ
# NULLはカウントしないからcountに差がでるよ(ex. スケジュールされたけど出発しなかった場合など)
traindata.describe().show()

+-------+----------+--------------+------------------+-------+------------------+------------------+---------------------+---------------------+------+------------------+-------------------+-------------------+-----+-------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-----------------+--------------------+------------------+------------------+------------------+--------------------+-----------------+------------------+--------------------+-----+-----------+
|summary|   FL_DATE|UNIQUE_CARRIER|        AIRLINE_ID|CARRIER|            FL_NUM| ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|   DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID| DEST|       CRS_DEP_TIME|           DEP_TIME|         DEP_DELAY|          TAXI_OUT|         WHEELS_OFF|          WHEELS_ON|           TAXI_IN|       CRS_ARR_TIME

In [13]:
# じゃあNULLを除外して特殊なフライトをなくそう
traindayquery = """
select
  f.*
from flights f
join traindays t
on f.FL_DATE == t.FL_DATE
where
  t.is_train_day == 'True' and
  f.dep_delay is not NULL and
  f.arr_delay is not NULL

"""
traindata = spark.sql(traindayquery)
traindata.describe().show()

+-------+----------+--------------+------------------+-------+------------------+------------------+---------------------+---------------------+------+------------------+-------------------+-------------------+-----+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-----------------+-------------------+-------------------+-------------------+---------+-----------------+--------+------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+-----+-----------+
|summary|   FL_DATE|UNIQUE_CARRIER|        AIRLINE_ID|CARRIER|            FL_NUM| ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|   DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID| DEST|       CRS_DEP_TIME|           DEP_TIME|         DEP_DELAY|          TAXI_OUT|         WHEELS_OFF|          WHEELS_ON|          TAXI_IN|       CRS_ARR_TIME|           ARR_TIME|   

 -> すべてのcount数が同じになった
 ただ根本解決にはなってない
 -> キャンセル、迂回が発生したかを表す絡むを見てレコードを絞った方が確実

In [14]:
query = """
select
  cancelled, diverted
from flights
limit 10
"""
spark.sql(query).show()

query = """
select
  cancelled, diverted
from flights
where
  cancelled != '0.00'
limit 10
"""
spark.sql(query).show()

query = """
select
  cancelled, diverted
from flights
where
  diverted != '0.00'
limit 10
"""
spark.sql(query).show()

+---------+--------+
|cancelled|diverted|
+---------+--------+
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
|     0.00|    0.00|
+---------+--------+

+---------+--------+
|cancelled|diverted|
+---------+--------+
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
|     1.00|    0.00|
+---------+--------+

+---------+--------+
|cancelled|diverted|
+---------+--------+
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
|     0.00|    1.00|
+---------+--------+



どうやらキャンセルされたか、迂回されたかは0,1で格納されているみたい

In [12]:
# もっとしっかり取り除こう
traindayquery = """
select
  f.*
from flights f
join traindays t
on f.FL_DATE == t.FL_DATE
where
  t.is_train_day == 'True' and
  f.cancelled == '0.00' and
  f.diverted == '0.00'
"""
traindata = spark.sql(traindayquery)
traindata.describe().show()

+-------+----------+--------------+------------------+-------+------------------+------------------+---------------------+---------------------+------+------------------+-------------------+-------------------+-----+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-----------------+-------------------+-------------------+-------------------+---------+-----------------+--------+------------------+------------------+------------------+--------------------+------------------+------------------+--------------------+-----+-----------+
|summary|   FL_DATE|UNIQUE_CARRIER|        AIRLINE_ID|CARRIER|            FL_NUM| ORIGIN_AIRPORT_ID|ORIGIN_AIRPORT_SEQ_ID|ORIGIN_CITY_MARKET_ID|ORIGIN|   DEST_AIRPORT_ID|DEST_AIRPORT_SEQ_ID|DEST_CITY_MARKET_ID| DEST|       CRS_DEP_TIME|           DEP_TIME|         DEP_DELAY|          TAXI_OUT|         WHEELS_OFF|          WHEELS_ON|          TAXI_IN|       CRS_ARR_TIME|           ARR_TIME|   

NULLを除外した時と同じ結果が得られた

# トレーニング

トレーニングデータの各レコードは、LabeldedPointクラスに変換する必要がある
- https://spark.apache.org/docs/latest/api/java/org/apache/spark/ml/classification/LogisticRegressionModel.html

In [4]:
!export PYSPARK_PYTHON=python3.5
!PYSPARK_DRIVER_PYTHON=python3.5
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint

In [5]:
def to_example(fields):
  return LabeledPoint(\
           float(fields['ARR_DALAY'] < 15), # on-time? \
           [\
             fields['DEP_DELAY'], \
             fields['TAXI_OUT'], \
             fields['DISTANCE'], \
           ])

In [6]:
# トレーニング用のラベルと入力変数だけを取り出したデータ
examples = traindata.rdd.map(to_example)

NameError: name 'traindata' is not defined

### トレーニング実施

In [3]:
# intercept: 切片
lrmodel = LogisticRegressionWithLBFGS.train(examples, intercept=True)
print(lrmodel.weights, lrmodel.intercept)

NameError: name 'LogisticRegressionWithLBFGS' is not defined